In [ ]:
# importing the dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Projects/SMS Classification using Transformer/SMSSpamCollection', sep = '\t',
                 names = ['Label', 'Message'])
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# making the X list
X = list(df['Message'])

In [ ]:
# making the y list
y = list(df['Label'])

In [ ]:
y[:10]

['ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'spam']

In [ ]:
# creating the dummies
y = list(pd.get_dummies(y, drop_first = True)['spam'])

In [ ]:
y[:10]

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [ ]:
len(y)

5572

In [ ]:
# splitting into train - test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
!pip install transformers

In [ ]:
# for every model of tranformer we have to take its own pretrained tokenizer 
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation = True, padding = True)
test_encodings = tokenizer(X_test, truncation = True, padding = True)

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir = '/content/drive/MyDrive/Projects/SMS Classification using Transformer/results', # output directory
    num_train_epochs = 5,                           # total number of epochs
    per_device_train_batch_size = 4,                # training batch size
    per_device_eval_batch_size = 8,                 # test batch size, will always be double of train, if this = 16, train = 8
    warmup_steps = 500,                             # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,                            # strength of weoght decay
    logging_dir = '/content/drive/MyDrive/Projects/SMS Classification using Transformer/logs', # directory for storing logs
    logging_steps = 10
)

In [ ]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = TFTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_59', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.04627663553046268}

In [ ]:
y_pred = trainer.predict(test_dataset)

In [ ]:
output = y_pred[1]

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, output)
cm

array([[1448,    0],
       [   0,  224]])

In [ ]:
trainer.save_model('/content/drive/MyDrive/Projects/SMS Classification using Transformer/senti_model')